# 청경채 잎 면적 증감률 예측

In [154]:
# 모듈 로딩
import pandas as pd
import numpy as np
import os
import datetime

In [134]:
# 훈련 데이터 경로
path='./data/train_input/'

In [135]:
# 파일 불러오기
df=pd.read_csv(path+'CASE_37.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63360 entries, 0 to 63359
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시간             63360 non-null  object 
 1   내부온도관측치        62953 non-null  float64
 2   내부습도관측치        62953 non-null  float64
 3   CO2관측치         62953 non-null  float64
 4   EC관측치          62953 non-null  float64
 5   외부온도추정관측치      0 non-null      float64
 6   외부습도추정관측치      0 non-null      float64
 7   펌프상태           62953 non-null  float64
 8   펌프작동남은시간       62953 non-null  float64
 9   최근분무량          62953 non-null  float64
 10  일간누적분무량        62953 non-null  float64
 11  냉방상태           62953 non-null  float64
 12  냉방작동남은시간       62953 non-null  float64
 13  난방상태           62953 non-null  float64
 14  난방작동남은시간       62953 non-null  float64
 15  내부유동팬상태        62953 non-null  float64
 16  내부유동팬작동남은시간    62953 non-null  float64
 17  외부환기팬상태        62953 non-null  float64
 18  외부환기팬작

In [126]:
# df=df.dropna()

In [136]:
df.columns=['시간', '내부온도관측치', '내부습도관측치', 'CO2관측치', 'EC관측치', '외부온도관측치', '외부습도관측치',
       '펌프상태', '펌프작동남은시간', '최근분무량', '일간누적분무량', '냉방상태', '냉방작동남은시간', '난방상태',
       '난방작동남은시간', '내부유동팬상태', '내부유동팬작동남은시간', '외부환기팬상태', '외부환기팬작동남은시간',
       '화이트 LED상태', '화이트 LED작동남은시간', '화이트 LED동작강도', '레드 LED상태', '레드 LED작동남은시간',
       '레드 LED동작강도', '블루 LED상태', '블루 LED작동남은시간', '블루 LED동작강도', '카메라상태', '냉방온도',
       '난방온도', '기준온도', '난방부하', '냉방부하', '총추정광량', '백색광추정광량', '적색광추정광량',
       '청색광추정광량']

In [128]:
# df[(df['내부유동팬상태']!=0)|(df['내부유동팬작동남은시간']!=0)][['내부유동팬상태','내부유동팬작동남은시간']]

,내부유동팬상태,내부유동팬작동남은시간


In [137]:

df1=df[['시간', '내부온도관측치', '내부습도관측치', 'CO2관측치', 'EC관측치',
        '일간누적분무량', '백색광추정광량', '적색광추정광량', '청색광추정광량']]

# '외부환기팬상태', '외부환기팬작동남은시간', '펌프상태', '펌프작동남은시간', '최근분무량', '냉방작동남은시간', '난방작동남은시간',  '내부유동팬상태', '내부유동팬작동남은시간', 
# '화이트 LED상태', '화이트 LED작동남은시간', '화이트 LED동작강도', '레드 LED상태', '레드 LED작동남은시간','레드 LED동작강도', '블루 LED상태', '블루 LED작동남은시간', '블루 LED동작강도', '카메라상태', 
# '난방부하', '냉방부하', '총추정광량', '냉방상태', '난방상태', '냉방온도', '난방온도', ,'기준온도' , '외부온도관측치', '외부습도관측치',

df1.describe()
#  

,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,일간누적분무량,백색광추정광량,적색광추정광량,청색광추정광량
count,62953.000000,62953.000000,62953.000000,62953.000000,0.0,0.0,62953.000000,62953.000000,62953.000000,62953.000000
mean,22.061640,63.757380,568.183639,2.088697,NaN,NaN,777.226824,145.455292,12.274693,20.984252
std,3.726251,12.700319,215.654256,1.578557,NaN,NaN,650.900535,148.009508,13.976560,23.976381
min,15.400000,21.700001,275.800000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,18.900000,60.000000,429.000000,0.000000,NaN,NaN,230.110000,0.000000,0.000000,0.000000
50%,20.900000,67.199997,493.000000,2.951732,NaN,NaN,659.550000,92.823000,6.619200,4.699500
75%,25.900000,73.099998,623.000000,3.294672,NaN,NaN,1135.090000,309.410000,23.167200,48.561500
max,37.500000,97.450002,1432.000000,4.285300,NaN,NaN,3306.920000,5825.000000,47.989200,61.093500


In [114]:
# df[(df['외부환기팬상태']!=0)|(df['외부환기팬작동남은시간']!=0)][['외부환기팬상태', '외부환기팬작동남은시간']]

#df[(df['백색광추정광량']!=0)|(df['적색광추정광량']!=0)|(df['청색광추정광량']!=0)][['백색광추정광량','적색광추정광량','청색광추정광량']]

In [115]:
df1['시간']=df1['시간'].apply(lambda x : x[0:10])
df1

C:\Users\User\AppData\Local\Temp\ipykernel_15296\202683288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['시간']=df1['시간'].apply(lambda x : x[0:10])


,시간,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,일간누적분무량,백색광추정광량,적색광추정광량,청색광추정광량
0,2022-02-24,10.6,50.700001,515.0,1.668448,5.000000,29.100000,0.00,6.1882,1.6548,1.5665
1,2022-02-24,10.5,50.900002,522.0,1.668448,4.900000,29.100000,389.63,6.1882,1.6548,1.5665
2,2022-02-24,10.4,51.299999,519.0,1.666178,4.900000,29.100000,389.63,6.1882,1.6548,1.5665
3,2022-02-24,10.3,51.099998,520.0,1.665042,4.800000,28.700001,389.63,0.0000,0.0000,0.0000
4,2022-02-24,10.3,51.099998,519.0,1.666178,4.800000,28.700001,389.63,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
31675,2022-03-17,16.4,84.300003,545.0,0.806695,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665
31676,2022-03-17,16.4,84.400002,549.0,0.806695,17.799999,40.700001,9136.61,12.3764,3.3096,1.5665
31677,2022-03-17,16.4,84.400002,553.0,0.806695,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665
31678,2022-03-17,16.4,84.400002,555.0,0.805838,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665


In [116]:
# 일별 최종 분무량만 남기기
# water=df[df['시간'].str.contains('23:59')][['시간','일간누적분무량']]
# water
df1

,시간,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,일간누적분무량,백색광추정광량,적색광추정광량,청색광추정광량
0,2022-02-24,10.6,50.700001,515.0,1.668448,5.000000,29.100000,0.00,6.1882,1.6548,1.5665
1,2022-02-24,10.5,50.900002,522.0,1.668448,4.900000,29.100000,389.63,6.1882,1.6548,1.5665
2,2022-02-24,10.4,51.299999,519.0,1.666178,4.900000,29.100000,389.63,6.1882,1.6548,1.5665
3,2022-02-24,10.3,51.099998,520.0,1.665042,4.800000,28.700001,389.63,0.0000,0.0000,0.0000
4,2022-02-24,10.3,51.099998,519.0,1.666178,4.800000,28.700001,389.63,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
31675,2022-03-17,16.4,84.300003,545.0,0.806695,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665
31676,2022-03-17,16.4,84.400002,549.0,0.806695,17.799999,40.700001,9136.61,12.3764,3.3096,1.5665
31677,2022-03-17,16.4,84.400002,553.0,0.806695,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665
31678,2022-03-17,16.4,84.400002,555.0,0.805838,17.900000,40.700001,9136.61,12.3764,3.3096,1.5665


In [117]:
df1=df1.groupby('시간')

In [118]:
df_max=df1['일간누적분무량'].max()

In [119]:
df_mean=df1.mean()

In [120]:
df_mean['일간누적분무량']=df_max
df_mean

,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,일간누적분무량,백색광추정광량,적색광추정광량,청색광추정광량
시간,,,,,,,,,,
2022-02-24,19.158457,51.434250,488.633194,1.641917,15.352847,19.948521,8467.89,193.727188,62.081431,39.259318
2022-02-25,22.385660,59.519632,480.117708,1.660300,21.393611,17.176333,9046.59,193.479015,62.009608,39.084175
2022-02-26,21.601389,68.821701,453.985069,1.687512,20.411493,23.827535,8632.47,193.586449,62.032591,39.100493
2022-02-27,22.476701,72.088792,428.821181,1.669893,21.491944,21.023507,8255.70,193.541327,61.983177,39.096141
2022-02-28,22.343493,74.003597,431.411250,1.661374,21.402118,21.499444,9953.22,193.664876,62.112458,39.103756
2022-03-01,23.234972,75.442226,431.110417,1.624008,22.239861,24.158125,8255.70,194.043118,62.117568,39.247352
2022-03-02,20.316354,76.854915,437.836389,1.601202,18.988681,24.363757,7898.22,194.055936,62.298623,39.227771
2022-03-03,21.082812,77.412587,441.157292,1.545096,19.952778,24.295000,7325.95,194.278324,62.249784,39.204926
2022-03-04,12.135737,80.060737,551.189103,12.886458,13.118590,29.555128,769.00,0.370234,0.116685,0.071965


In [121]:
# df_np=np.array(df1)
# df_np.shape

In [156]:
# 타겟 데이터 경로
path_target='./data/train_target/'

target_file=os.listdir(path)
target=pd.DataFrame()

for i in target_file:
    df=pd.read_csv(path+i)
    df['시간']=pd.to_datetime(df['시간'])+datetime.timedelta(days=-1)
    df['시간']=df['시간'].astype('object')
    df['시간']=df['시간'].apply(lambda x : str(x)[0:10])
    target=pd.concat([target, df], ignore_index=True)
target

,시간,rate
0,2021-02-17,0.50000
1,2021-02-18,0.66667
2,2021-02-19,0.60000
3,2021-02-20,-0.12500
4,2021-02-21,1.42857
...,...,...
1808,2022-06-10,-0.07888
1809,2022-06-11,-0.15838
1810,2022-06-12,-0.47374
1811,2022-06-13,-0.51767


In [146]:
len(target)

1813

In [150]:

path_target='./data/train_target/'
df=pd.read_csv(path_target+'CASE_01.csv')
df['시간']=df['시간'].apply(lambda x : x[0:10])

path_input='./data/train_input/'
df1=pd.read_csv(path_input+'CASE_01.csv')
df['시간']=df['시간'].apply(lambda x : x[0:10])
df

,시간,rate
0,2021-02-18,0.50000
1,2021-02-19,0.66667
2,2021-02-20,0.60000
3,2021-02-21,-0.12500
4,2021-02-22,1.42857
5,2021-02-23,0.35294
6,2021-02-24,0.30435
7,2021-02-25,0.16667
8,2021-02-26,0.22857
9,2021-02-27,0.32558


In [ ]:
merge_left = pd.merge(df1,df2, how='left', left_on='stock_name', right_on='name')
merge_left

In [ ]:
target_file=os.listdir(path)
target=pd.DataFrame()
for i in target_file:
    df=pd.read_csv(path+f'CASE_{i}.csv')
    print(df)

In [123]:
import reverse_geocode
coordinates = (-37.81, 144.96), (31.76, 35.21)
reverse_geocode.search(coordinates)

[{'country_code': 'AU', 'city': 'Melbourne', 'country': 'Australia'},
 {'country_code': 'IL', 'city': 'Jerusalem', 'country': 'Israel'}]